<a href="https://colab.research.google.com/github/badbloody/diploma2023/blob/main/fid_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# example of calculating the frechet inception distance in Keras for cifar10
import numpy
import numpy as np
import tensorflow as tf
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from keras.datasets import cifar10

# scale an array of images to a new size
def scale_images(images, new_shape):
 images_list = list()
 for image in images:
    # resize with nearest neighbor interpolation
    new_image = resize(image, new_shape, 0)

    # store
    images_list.append(new_image)
 return asarray(images_list)

# calculate frechet inception distance
def calculate_fid(model, images1, images2, batch_size=32):
  act1_list = []
  act2_list = []
  for i in range(0, len(images1), batch_size):
      batch_images1 = images1[i:i+batch_size]
      batch_images2 = images2[i:i+batch_size]

      act1_batch = model.predict(batch_images1)
      act2_batch = model.predict(batch_images2)

      act1_list.append(act1_batch)
      act2_list.append(act2_batch)

  act1 = np.concatenate(act1_list, axis=0)
  act2 = np.concatenate(act2_list, axis=0)

  # calculate mean and covariance statistics
  mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
  mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)

  # calculate sum squared difference between means
  ssdiff = numpy.sum((mu1 - mu2)**2.0)

  # calculate sqrt of product between cov
  covmean = sqrtm(sigma1.dot(sigma2))

  # check and correct imaginary numbers from sqrt
  if iscomplexobj(covmean):
    covmean = covmean.real

  # calculate score
  fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
  return fid

model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

# load cifar10 images
(images1, _), (images2, _) = cifar10.load_data()
shuffle(images1)
shuffle(images2)
images1 = images1[:10]
images2 = images2[:10]
print('Loaded', images1.shape, images2.shape)

# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')

# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)

# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)

batch_size = 32
fid = calculate_fid(model, images1, images2, batch_size)
print('FID: %.3f' % fid)

# Clear TensorFlow session to release GPU memory
tf.keras.backend.clear_session()

Loaded (10, 32, 32, 3) (10, 32, 32, 3)
Scaled (10, 299, 299, 3) (10, 299, 299, 3)
1/1 [==============================] - 0s 34ms/step
FID: 361.146
